![En tête general](img/En_tete_general.png)


*(C) Copyright Franck CHEVRIER 2019-2020 http://www.python-lycee.com/*

<span style="color: #9317B4"> Pour exécuter une saisie Python, sélectionner la cellule et valider avec </span><span style="color: #B317B4"><strong>SHIFT+Entrée</strong></span>.

# Exerciseur : Calculs de dérivées

In [ ]:

# GENERATION DES LISTES DE FONCTIONS USUELLES




f_trigo         = [ 'cos(x)','sin(x)' ]
f_puissance_pos = [ 'x**'+str(k) for k in range(1,4) ]
f_puissance_neg = [ 'x**'+str(k) for k in range(-3,0) ]
f_racine        = [ 'sqrt(x)']
f_exp           = [ 'exp(x)' ]
f_ln            = [ 'ln(x)' ]

def f_polynome(deg, withrat=True):
    """
    génère un polynôme
    deg : degré du polynôme
    withrat : autorise les coefficients rationnels (entiers sinon)
    """
    f = ""
    for k in range(deg+1):
        f += choice(["+","-"])+str(Rational(randint(0 if k<deg else 1,10),randint(1,10)) if withrat else randint(0 if k<deg else 1,10))+"*x**"+str(k)
    return parse_expr(f)





from sympy import *                                         # import du module sympy
from random import randint,choice                           # import des fonctions générant l'aléatoire
import IPython ; from IPython.display import display, HTML  # pour la génération des écritures latex en htmlµ
from time import sleep

x=symbols('x')






def par(expr,gauche=True):
    """
    génère une parenthèse latex si l'expression démarre par -
    ( si gauche=True
    ) si gauche= False
    """
    return "" if expr[0]!="-" else ("\\left( " if gauche else "\\right) ")

def sig(expr):
    """
    génère un + si l'expression ne démarre pas par -
    """
    return "" if expr[0]=="-" else "+"

def rempl(expr):
    """
    remplace log par ln dans une chaine
    remplace les puissances à coeff 1/2 par des racines carrées
    """
    expr=expr.replace("log","ln")
    for k in range(20):
            expr=expr.replace("x^{\\frac{"+str(k)+"}{2}}","x^{"+str(k//2)+"} \\sqrt{x} ")
            expr=expr.replace("x^{"+str(k)+".5}","x^{"+str(k)+"} \\sqrt{x} ")
    return expr



# LES FONCTIONS QUI GENERENT UN EXERCICE (càd UNE FONCTION A DERIVER)
# RECOIVENT EN ARGUMENT UNE LISTE DE LISTES DE FONCTIONS
# (UNIFORMISATION POUR L'APPEL)







def derivee_produit_par_scalaire(l=[liste_fonc]):
    """
    génère une fonction produit d'un coefficient rationnel et d'une fonction usuelle
    renvoie :
    le titre de l'exercice
    la consigne (str)
    la solution
    la liste des détails de calcul (liste de str)
    la liste des feedback (juste, faux 1er essai, faux 2e essai) chacun de type liste de str
    la liste des lignes d'aide (liste de str)
    """
    Coeff = choice([-1,1])*Rational(randint(1,10),randint(1,10)) # génération d'un coefficient fractionnaire
    f_usuelle = eval(choice(l[0])) # génération d'une fonction usuelle
    
    f = Coeff*f_usuelle 
    
    correction=["f(x)="+latex(f)]
    
    etape = "f(x)="+latex(Coeff)+"\\times "+latex(f_usuelle)
    if etape!=correction[-1]: correction.append(etape)
    
    correction.append(" \\text{ a pour dérivée : }")
    
    f_usuelle_der = simplify(diff(f_usuelle,x))
    etape = "f'(x)="+latex(Coeff)+"\\times "+par(latex(f_usuelle_der))+latex(f_usuelle_der)+par(latex(f_usuelle_der),False)
    correction.append(etape) 
    
    etape = "f'(x)="+latex(Coeff*f_usuelle_der)
    if etape!=correction[-1]: correction.append(etape)
    
    sol = diff(f,x)
    etape = "f'(x)="+latex(sol)
    if etape!=correction[-1]: correction.append(etape)
    
    sol=factor(sol)
    etape = "f'(x)="+latex(sol)
    if etape!=correction[-1]: correction.append(etape)
    
    sol=simplify(sol)
    etape = "f'(x)="+latex(sol)
    if etape!=correction[-1]: correction.append(etape)    
    
    consigne = "<latex> \[ \\text{Donner l'expression de la dérivée de } f \\text{ définie par } f(x)= "+latex(f)+"  \\text{.} \] </latex>"
    
    L_feedback = [  [" \\color{#239B56}{\\text{Réponse correcte.}}",
                     " \\color{#239B56}{\\text{La réponse attendue était } f'(x)="+latex(sol)+"\\text{.}}"
                    ],
                    [" \\color{#D35400}{\\text{Réponse fausse.}}",
                     " \\color{#D35400}{\\text{Une aide est disponible pour le 2}^{\\text{ème}} \\text{ essai.}}"
                    ],
                    [" \\color{#E71212}{\\text{Réponse fausse.}}",
                     " \\color{#E71212}{\\text{La réponse attendue était } f'(x)="+latex(sol)+"\\text{. Voir correction ci-dessus.}}"
                    ]
                    ]
                        

    L_aide = ["\\text{ La formule à appliquer est :} ",
                "\\; \\left( k \\times u \\right)' = k \\times u' \\;",
                "\\text{ où } k \\text{ constante et } u \\text{ fonction dérivable. }  "
              ]
    
    return consigne,sol,correction,L_feedback,L_aide


    
def derivee_somme(l=[liste_fonc,liste_fonc]):
    """
    génère une fonction somme de deux fonctions usuelles (avec coefficients multiplicatifs rationnels)
    renvoie :
    le titre de l'exercice
    la consigne (str)
    la solution
    la liste des détails de calcul (liste de str)
    la liste des feedback (juste, faux 1er essai, faux 2e essai) chacun de type liste de str
    la liste des lignes d'aide (liste de str)
    """
    Coeff1 = choice([-1,1])*Rational(randint(1,10),randint(1,10)) # génération d'un coefficient fractionnaire    
    f_usuelle1 = eval(choice(l[0])) # génération d'une fonction usuelle
    
    Coeff2 = choice([-1,1])*Rational(randint(1,10),randint(1,10)) # génération d'un coefficient fractionnaire    
    
    f_usuelle2 = f_usuelle1
    while f_usuelle2==f_usuelle1: # génération d'une AUTRE fonction usuelle
        f_usuelle2 = eval(choice(l[1]))     
    
    f = Coeff1*f_usuelle1+Coeff2*f_usuelle2 
    
    correction=["f(x)="+latex(f)]
    
    etape="f(x)="+latex(Coeff1)+"\\times "+latex(f_usuelle1)+sig(latex(Coeff2))+latex(Coeff2)+"\\times "+latex(f_usuelle2)
    if etape!=correction[-1]: correction.append(etape)
    
    correction.append(" \\text{ a pour dérivée : }")
    
    f_usuelle1_der = simplify(diff(f_usuelle1,x))
    f_usuelle2_der = simplify(diff(f_usuelle2,x))
    
    etape="f'(x)="+latex(Coeff1)+"\\times "+par(latex(f_usuelle1_der))+latex(f_usuelle1_der)+par(latex(f_usuelle1_der),False)+sig(latex(Coeff2))+latex(Coeff2)+"\\times "+par(latex(f_usuelle2_der))+latex(f_usuelle2_der)+par(latex(f_usuelle2_der),False)
    if etape!=correction[-1]: correction.append(etape)
    
    #etape="f'(x)="+latex(Coeff1*f_usuelle1_der)+"+"+("\\left( " if latex(Coeff2*f_usuelle2_der)[0]=="-" else "")+latex(Coeff2*f_usuelle2_der)+("\\right) " if latex(Coeff2*f_usuelle2_der)[0]=="-" else "")
    #if etape!=correction[-1]: correction.append(etape)
    
    etape="f'(x)="+latex(simplify(Coeff1*f_usuelle1_der+Coeff2*f_usuelle2_der))
    if etape!=correction[-1]: correction.append(etape)
    
    sol=simplify(diff(f,x))
    etape="f'(x)="+latex(sol)
    if etape!=correction[-1]: correction.append(etape)
    
    sol=factor(sol)
    etape="f'(x)="+latex(sol)
    if etape!=correction[-1]: correction.append(etape)    
    
    consigne = "<latex> \[ \\text{Donner l'expression de la dérivée de } f \\text{ définie par } f(x)= "+latex(f)+"  \\text{.} \] </latex>"
    
    L_feedback = [  [" \\color{#239B56}{\\text{Réponse correcte.}}",
                     " \\color{#239B56}{\\text{La réponse attendue était } f'(x)="+latex(sol)+"\\text{.}}"
                    ],
                    [" \\color{#D35400}{\\text{Réponse fausse.}}",
                     " \\color{#D35400}{\\text{Une aide est disponible pour le 2}^{\\text{ème}} \\text{ essai.}}"
                    ],
                    [" \\color{#E71212}{\\text{Réponse fausse.}}",
                     " \\color{#E71212}{\\text{La réponse attendue était } f'(x)="+latex(sol)+"\\text{. Voir correction ci-dessus.}}"
                    ]
                    ]
                        

    L_aide = ["\\text{ La formule à appliquer est :} ",
                "\\; \\left( u + v \\right)' =  u' + v' \\;",
                "\\text{ où } u \\text{ et } v \\text{ fonctions dérivables. }  "
             ]
    
    return consigne,sol,correction,L_feedback,L_aide



    
    
    
    
    
    
    
def derivee_produit(l=liste_fonc):
    """
    génère une fonction produit d'un polynôme (avec coefficients multiplicatifs rationnels) et d'une fonction usuelle
    renvoie :
    le titre de l'exercice
    la consigne (str)
    la solution
    la liste des détails de calcul (liste de str)
    la liste des feedback (juste, faux 1er essai, faux 2e essai) chacun de type liste de str
    la liste des lignes d'aide (liste de str)
    """
    
    Coeff1 = choice([-1,1])*Rational(randint(1,10),randint(1,10)) # génération d'un coefficient fractionnaire    
    # le coeff est rattaché à la fonction 1 
    
    f_usuelle1 = eval(choice(l[0])) # génération d'une fonction usuelle
        
    f_usuelle2 = eval(choice(l[1])) # génération d'un polynôme
     
    f = (Coeff1*f_usuelle1)*f_usuelle2 
        
    correction=["f(x)="+latex(f)]
    
    etape="f(x)="+latex(simplify(Coeff1*f_usuelle1))+" \\times "+latex(f_usuelle2)
    if etape!=correction[-1]: correction.append(etape)
    
    correction.append(" \\text{ a pour dérivée : }")
    
    f_usuelle1_der = simplify(diff(f_usuelle1,x))
    f_usuelle2_der = simplify(diff(f_usuelle2,x))
    
    etape="f'(x)="+latex(simplify(Coeff1*f_usuelle1_der))+" \\times "+latex(f_usuelle2)+"+"+latex(simplify(Coeff1*f_usuelle1))+" \\times "+latex(f_usuelle2_der)
    if etape!=correction[-1]: correction.append(etape)
    
    etape="f'(x)="+latex(simplify(Coeff1*f_usuelle1_der*f_usuelle2))+"+"+latex(simplify(Coeff1*f_usuelle1*f_usuelle2_der))
    if etape!=correction[-1]: correction.append(etape)
    
    etape="f'(x)="+latex(simplify(Coeff1*f_usuelle1_der*f_usuelle2+Coeff1*f_usuelle1*f_usuelle2_der))
    if etape!=correction[-1]: correction.append(etape)
    
    sol=simplify(diff(f,x))
    etape="f'(x)="+latex(sol)
    if etape!=correction[-1]: correction.append(etape)                     
                         
    sol=factor(sol)
    etape="f'(x)="+latex(sol)
    if etape!=correction[-1]: correction.append(etape)    
    
    consigne = "<latex> \[ \\text{Donner l'expression de la dérivée de } f \\text{ définie par } f(x)= "+latex(simplify(Coeff1*f_usuelle1))+" \\times "+latex(f_usuelle2)+"  \\text{.} \] </latex>"
    
    L_feedback = [  [" \\color{#239B56}{\\text{Réponse correcte.}}",
                     " \\color{#239B56}{\\text{La réponse attendue était } f'(x)="+latex(sol)+"\\text{.}}"
                    ],
                    [" \\color{#D35400}{\\text{Réponse fausse.}}",
                     " \\color{#D35400}{\\text{Une aide est disponible pour le 2}^{\\text{ème}} \\text{ essai.}}"
                    ],
                    [" \\color{#E71212}{\\text{Réponse fausse.}}",
                     " \\color{#E71212}{\\text{La réponse attendue était } f'(x)="+latex(sol)+"\\text{. Voir correction ci-dessus.}}"
                    ]
                    ]
                        

    L_aide = ["\\text{ La formule à appliquer est :} ",
                "\\; \\left( u \times v \\right)' =  u' \times v + u \times v' \\;",
                "\\text{ où } u \\text{ et } v \\text{ fonctions dérivables. }  "
             ]
    
    return consigne,sol,correction,L_feedback,L_aide

















def html_presentation(titre_exercice="",decompte_reponses=[0,0,0],insert_html='',align='center',width=800,height=70,backcolor='#FAE5D3',padding=0.5,margin=0.1):
    """
    création de la boîte du titre 
    """   
    insert_html += '<div id="presentation" style="float: '+align+'; width: '+str(width)+'px; height: '+str(height)+'px; background-color: '+backcolor+'; padding:'+str(padding)+'em; ;margin: '+str(margin)+'em;">'
    
    # score
    
    insert_html += """
    
    <table style="float: right;">
    <tr style="padding=0.1em; background-color:#EDBB99; ">
        <td style="text-align:left; font-size: 14px; text-align:center;"><strong>Score :</strong></td>
        <td style="text-align:left; font-size: 14px; text-align:center;"><strong>
    """    
    
    if decompte_reponses[0]!=0:
        doublenote=2*decompte_reponses[1]+decompte_reponses[2]
        pair=doublenote%2==0
        insert_html += "<latex> \\[ \\frac{"+str((doublenote)//2)+("" if pair else ",5")+"}{"+str(decompte_reponses[0])+"} \\] </latex>"
    else:
        insert_html += "Pas de note"
        
    insert_html += """    
        </strong></td>
    </tr>
    <tr style="padding=0.1em; background-color:#F6DDCC;">
        <td style="text-align:left; font-size: 10px; text-align:left;">Juste 1<sup>er</sup> essai :</td>
        <td style="text-align:left; font-size: 10px; text-align:center;">
    """ 
    insert_html += str(decompte_reponses[1])
    insert_html += """    
        </td>
    </tr>
    <tr style="padding=0.1em; background-color:#F6DDCC;">
        <td style="text-align:left; font-size: 10px; text-align:left;">Juste 2<sup>ème</sup> essai :</td>
        <td style="text-align:left; font-size: 10px; text-align:center;">
    """
    insert_html += str(decompte_reponses[2])
    insert_html += """    
        </td>
    </tr>
    <tr style="padding=0.1em; background-color:#F6DDCC;">
        <td style="text-align:left; font-size: 10px; text-align:left;">Faux :</td>
        <td style="text-align:left; font-size: 10px; text-align:center;">
    """    
    insert_html += str(decompte_reponses[0]-(decompte_reponses[1]+decompte_reponses[2]))   
    insert_html += """     
        </td>
    </tr>
    </table>
    """
        
    # titre
    insert_html += '<p>'
    insert_html += '<span style="color:#123456; font-size: 8px;  text-align:center;">&copy www.python-lycee.com</span></br>'
    insert_html += '<span style="color:#123456; font-size: 16px; text-align:center;"><strong>Exercice : '+titre_exercice+'</strong></span>'
    insert_html += '</p>'
        
    insert_html += '</div>'
    
    
    return insert_html

def html_consigne(entete="",consigne="",insert_html='',align='center',width=800,height=75,backcolor='#FDF2E9',padding=0.5,margin=0.1):
    """
    création de la boîte de consigne
    entete,consigne de type str
    """
    insert_html += '<div id="consigne" style="float: '+align+'; width: '+str(width)+'px; height: '+str(height)+'px; background-color: '+backcolor+'; padding:'+str(padding)+'em; ;margin: '+str(margin)+'em;">'
    insert_html += '<h2 style="color:#123456; font-size: 14px; text-align:left;"><strong>'+entete+'</strong><br>'
    insert_html += rempl(consigne)+'</h2>'
    insert_html += '</div>'
    return insert_html

def html_correction(entete="",correction="",vide=False,insert_html='',align='center',width=800,height=350,backcolor='#FDF2E9',padding=0.5,margin=0.1):
    """
    création de la zone de correction
    entete de type str
    correction : liste des lignes de type str
    vide : booléen qui indique si la zone doit être vide
    """
    insert_html += '<div id="correction" style="float: '+align+'; min-width: '+str(width)+'px; max-width: '+str(width)+'px; width: '+str(width)+'px; height: '+str(height)+'px;  background-color: '+backcolor+'; padding:'+str(padding)+'em; ;margin: '+str(margin)+'em;">'
    if not vide: 
        insert_html += '<h2 style="color:#123456; font-size: 14px; text-align:left;"><strong>'+entete+'</strong><br>'
        for ligne in correction:
            insert_html += '<latex> \[ '+rempl(ligne)+' \] </latex> <br>'
        insert_html += '</h2>'
    insert_html += '</div>'
    return insert_html

    # overflow-x:scroll; overflow-y:scroll; window.document.getElementById(div).scrollTop = window.document.getElementById(div).scrollHeight;


def html_feedback(commentaires=[],aide=None,insert_html='',align='center',width=800,height=125,backcolor='#FAE5D3',padding=0.5,margin=0.1):
    """
    création de la zone de feedback
    feedback : liste des lignes de type str
    """
    insert_html += '<div id="correction" style="float: '+align+'; min-width: '+str(width)+'px; max-width: '+str(width)+'px; width: '+str(width)+'px; height: '+str(height)+'px;  background-color: '+backcolor+'; padding:'+str(padding)+'em; ;margin: '+str(margin)+'em;">'
    
    #affichage de l'aide
    if aide:
        

        insert_html += """
        
        <div class="Aide_parent">
        <span style="color:#14A752; font-weight:bold;"> Aide </span>
        <div class="Aide_enfant">
        <div style="color:#14A752; text-align: left; background-color: #D9FEEC;">
        """
        for ligne in aide:
            insert_html += '<latex> \[ '+rempl(ligne)+' \] </latex>' 
        insert_html += """
        </div>
        </div>
        </div>
        
        """  
    else:
        insert_html += """
        <div class="Aide_parent">
        <span style="color:#888888;"> Aide </span>
        <div class="Aide_enfant">
        <span style="font-size: 10px; color:#888888;">L'aide est accessible pour le 2ème essai</span>
        </div>
        </div>
        
        """

    
    insert_html += '<h2 style="color:#123456; font-size: 14px; text-align:left;">'
    for ligne in commentaires:
        insert_html += '<latex> \[ '+rempl(ligne)+' \] </latex>'
    insert_html += '</h2>'
    insert_html += '</div>'
    return insert_html

    # overflow-x:scroll; overflow-y:scroll; window.document.getElementById(div).scrollTop = window.document.getElementById(div).scrollHeight;

  
    

    
def Genere_html(exercice,consigne,correction,commentaires=[],vide=False,decompte_reponses=[0,0,0],aide=None,titre_exercice=""):
    """
    Fonction qui génère l'affichage html
    """   
    body = "<body>"
    body += html_presentation(titre_exercice=titre_exercice,decompte_reponses=decompte_reponses) #insertion du titre
    body += html_consigne(entete="Consigne :",consigne=consigne) #insertion de la consigne
    body += html_correction(entete="Correction :",correction=correction,vide=vide) #insertion de la zone de correction
    body += html_feedback(commentaires,aide)

    body += "</body>"    
    return body












# GENERATION DES LISTES D'EXERCICES

exercices = [ (derivee_produit_par_scalaire,[liste_fonc]),(derivee_somme,[liste_fonc3,liste_fonc3])]    

exercices2 = [ (derivee_produit,[liste_fonc,liste_fonc]) ]


#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html




def Exerciceur_Derivation(exercices=exercices,liste_fonc=liste_fonc,titre_exercice="Calculs de dérivées"): 
    """
    Argument exercice : fonction qui génère l'exercice
    
    """
    
    #création du head de la page html avec insertions css
    head = """
    <head>
        <meta charset="utf-8" />
        <title> Exerciseur python-lycee.com </title>
        <script type="text/javascript" src="http://latex.codecogs.com/latexit.js"></script>
        <script type="text/javascript">LatexIT.add('latex',true);</script>
        <style>
        

        
        div.Aide_enfant
            {
             display: none;
             
            }

        div.Aide_parent:hover .Aide_enfant
            {
            display:block;
            }
        div.Aide_parent
            {
            text-align: right;
            float: right;
            }
        
        </style>
    </head>      
    """ #reprendre des élements de BBDSQL pour le style
    
    
    # Constantes pour le score
    decompte_reponses = [0,0,0] # nbre de questions traitées, nbre de juste en 1, nbre de juste en 2
    
    
    
    while "on ne quitte pas":  

        # génération de l'énoncé+aide+feedback
        # l'exercice est choisi au hasard parmi la liste des possibilités
        exercice,l = choice(exercices)
        consigne,sol,correction,L_feedback,L_aide = exercice(l=l) 
        
        
        
        
        # création de l'affichage html
        body = Genere_html(exercice,consigne,correction,vide=True,decompte_reponses=decompte_reponses,titre_exercice=titre_exercice)
                
        # raffraichissement de l'affichage
        IPython.display.clear_output(wait=True)
        raffraichi = None == display(HTML('<html>'+head+body+'</html>'))
        
        # précaution d'attente pour que le input de la requête soit inséré après le html
        while not raffraichi: None 
        sleep(0.2)                           
        
        
        for essai in [1,2]: # 2 essais possibles
            
            aide_dispo = False # on désactive l'aide
            
            # input de la proposition
            while True: 
                entree = input("\nVeuillez saisir et valider votre proposition :")
                try:
                    proposition = parse_expr(entree) ; break
                except:
                    print("Saisie invalide.")    
            
            # vérification de la proposition
            Verification = expand(proposition) == expand(sol)
            
            if Verification:        
                decompte_reponses[essai] += 1 # on incrémente le décompte correspond à l'essai concluant
                decompte_reponses[0] +=1      # on incrémente le compteur du nombre de questions
                
                commentaires = ["\\text{Votre saisie : }"+str(latex(proposition))]+L_feedback[0] # création du commentaire feedback (réponse juste)
                
            else:
                if essai==1 :                     # si erreur au 1er essai
                    aide_dispo = True            # on permet l'affichage de l'aide    
                
                if essai==2 :                     # si erreur au 2e essai
                    decompte_reponses[0] +=1      # on incrémente le compteur du nombre de questions       
                commentaires = ["\\text{Votre saisie : }"+str(latex(proposition))]+L_feedback[essai] # création du commentaire feedback (réponse fausse)
                
                
                
            # création de l'affichage html
            body = Genere_html(exercice,consigne,correction,commentaires=commentaires,vide=(essai==1 and not Verification),decompte_reponses=decompte_reponses,aide=(L_aide if aide_dispo else None),titre_exercice=titre_exercice) 
            
            # raffraichissement de l'affichage
            IPython.display.clear_output(wait=True)
            raffraichi = None == display(HTML('<html>'+head+body+'</html>'))
        
            # précaution d'attente pour que le input de la requête soit inséré après le html
            while not raffraichi: None 
            sleep(0.2)                           
            
            if Verification: 
                break
        
             
        # Attente de validation pour continuer ou quitter
        Choix = input("Valider pour continuer (pour quitter :saisir q ou Q et valider)")
        if Choix in ['q','Q']: break
    
    IPython.display.clear_output()       
    return None    
        
    
        


Exerciceur_Derivation(exercices2)

In [1]:
import random
from ipywidgets import IntSlider, Output, widgets
from IPython.display import clear_output
from IPython.display import display


Choix_prod_scal = widgets.Dropdown(
    options=['Sans', 'Avec fonctions simples', ]



    )

selectmode = "multiple"


accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text()], titles=('Slider', 'Text'))
accordion

dropdown = widgets.Dropdown(options=['button', 'slider'])
widgets.jslink((dropdown, 'index'), (stacked, 'selected_index'))
widgets.VBox([dropdown, stacked])

NameError: name 'stacked' is not defined

In [11]:
from ipywidgets import IntSlider, Output, widgets
from IPython.display import clear_output
from IPython.display import display
a=widgets.RadioButtons(
    options=['pepperoni', 'pineapple', 'anchovies'],
    #    value='pineapple', # Defaults to 'pineapple'
    #    layout={'width': 'max-content'}, # If the items' names are long
    selectmode = "multiple",
    description='Pizza topping:',
    disabled=False,
)
a

RadioButtons(description='Pizza topping:', options=('pepperoni', 'pineapple', 'anchovies'), value='pepperoni')

In [6]:
a.value

'pepperoni'

In [13]:
b=widgets.SelectMultiple(
    options=['Apples', 'Oranges', 'Pears'],
    value=['Oranges'],
    #rows=10,
    description='Fruits',
    disabled=False
)
b

SelectMultiple(description='Fruits', index=(1,), options=('Apples', 'Oranges', 'Pears'), value=('Oranges',))

In [32]:






#Menu Coefficients
Menu_Coefficients = widgets.SelectMultiple (
    description = "Coefficients",
    options=['Coefficients entiers', 'Coefficients rationnels'],
    value=['Coefficients rationnels'],
    #rows=10,
    disabled=False
    )    
#Menu Fonctions
Menu_Fonctions = widgets.SelectMultiple (
    description = "Fonctions utilisées",
    options=['puissances positives', 'puissances négatives', 'racine carrée', 'cosinus et sinus','exponentielle', 'logarithme népérien'],
    value=['puissances positives', 'puissances négatives', 'racine carrée'],
    #rows=10,
    disabled=False
    ) 
#Menu Formules utilisées
Menu_Formules = widgets.SelectMultiple (
    description = "Formules utilisées",
    options=['Produit par un scalaire', 'Somme de fonctions', 'Produit de fonctions', 'Quotient de fonctions','Composition par un polynôme'],
    value=['Produit par un scalaire', 'Somme de fonctions', 'Produit de fonctions', 'Quotient de fonctions'],
    #rows=10,
    disabled=False
    ) 
              
                         
                          
Menu_accordeon       

Accordion(children=(SelectMultiple(index=(0, 1), options=('Coefficients entiers', 'Coefficients rationnels'), …

In [33]:
accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text()], titles=('Slider', 'Text'))
accordion

Accordion(children=(IntSlider(value=0), Text(value='')))

In [2]:
import random
from ipywidgets import IntSlider, Output, widgets
from IPython.display import clear_output
from IPython.display import display


button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button)

out = Output()
display(out)

def on_button_clicked(b):
    with out:
        clear_output(True)
        print("Button clicked.",round(random.uniform(0, 1),3))
        
        
button.on_click(on_button_clicked)


Button(description='Click Me!', style=ButtonStyle())

Output()

In [ ]:
a.p

In [ ]:
a.ap()

In [ ]:
type(a.p)

In [ ]:
f=1
type(latex(f))

In [ ]:
f=x**2/(x+1)

In [ ]:
f

In [ ]:
f.p

In [ ]:
simplify(f).p

In [ ]:
f=(x+1)/(2*x+1) ; g=(3*x+7)/(2*x+1)**2

In [ ]:
f+g

In [ ]:
factor(f+g)

In [ ]:
sismplify(f+g)

In [ ]:
expand(simplify((f+g)))

In [ ]:
f+g

In [ ]:
chaine="log(10)"
chaine=chaine.replace("log","ln")

In [ ]:
chaine


In [ ]:
f=sqrt(x)

In [ ]:
from IPython.display import clear_output

for i in range(10000):
    clear_output(wait=True)
    print(i, flush=True)


In [ ]:
diff(f)

In [ ]:
diff(f*f)

In [ ]:
diff(f)*x

In [ ]:
f=1/sqrt(x)+1/x


In [ ]:
g=diff(f,x)

In [ ]:
g


In [ ]:
latex(g)

In [ ]:
a="djfeojfiejfijf"

In [ ]:
a.replace(["e","p"],["l","l"])

In [ ]:
f=x**(11/2)


In [ ]:
f

In [ ]:
latex(f)

In [ ]:
f = sqrt(x) + x**5


In [ ]:
g=diff(f,x)

In [ ]:
g

In [ ]:
simplify(g)

In [ ]:
g=factor(g)

In [ ]:
f=e**x


In [ ]:
latex(g)

In [ ]:
https://qastack.fr/programming/14824163/how-to-get-the-input-from-the-tkinter-text-widget
https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
https://stackoverflow.com/questions/43677564/passing-input-from-html-to-python-and-back

In [ ]:
rempl(latex(g))

In [ ]:
def rempl(expr):
    """
    remplace log par ln dans une chaine
    remplace les puissances à coeff 1/2 par des racines carrées
    """
    expr=expr.replace("log","ln")
    for k in range(20):
            expr.replace("x^{\\frac{"+str(k)+"}{2}}","x^{"+str(k//2)+"} sqrt{x} ")
            print("x^{\\frac{"+str(k)+"}{2}}")
            print("x^{\\frac{"+str(k)+"}{2}}" in expr,expr)
            expr.replace("x^{"+str(k)+".5}","x^{"+str(k)+"} \\sqrt{x} ")
    return expr


f = sqrt(x) + x**5 ; g=diff(f,x) ; g=factor(g)
latex(g) , rempl(latex(g))

In [ ]:
"x^{\\frac{9}{2}}" in "\\frac{10 x^{\\frac{9}{2}} + 1}{2 \\sqrt{x}}"

In [ ]:
getvalue('pay')

In [ ]:
import IPython ; from IPython.display import display, HTML


import ipywidgets as widgets

"""

turn_on = 'Run'
turn_off = 'Skip'

skipp = True

button_exec = widgets.Button(description=turn_on)
display(button_exec)

def exec_clicked(b):
    global skipp
    print(a.value)
    if skipp:
        b.description = turn_off
    else:
        b.description = turn_on
    skipp = not skipp

button_exec.on_click(exec_clicked)

"""



# boutons de choix

r=widgets.Dropdown(
    options=['Coefficients entiers', 'Coefficients rationnels'],
    value='Coefficients entiers',
    description='Coefficients',
    disabled=False,
)

widgets.ToggleButtons(
    options=['Slow', 'Regular', 'Fast'],
    description='Speed:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

a = widgets.Checkbox(
    value=False,
    description='fz$e^3$odfjk',
    disabled=False,
    indent=True,

)

a2 = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False,
    indent=True,

)

button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)




"""
display(a,a2)

def exec_clicked(b):
    IPython.display.clear_output()

    Derivation_dir_form(derivee_somme,liste_fonc3)
  
display(button)
button.on_click(exec_clicked)
#IPython.display.clear_output()
"""
h=widgets.Text()
display(h)

In [ ]:
a.packforget()

In [ ]:
Pour info:
Rendre un bouton invisible:Bouton.pack_forget()
Remettre un bouton visible:Bouton.pack()
Rendre un bouton inactif:Bouton.config(state = DISABLED)
Rendre un bouton actif:Bouton.config(state = NORMAL)
   
https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
http://tkinter.fdex.eu/doc/chw.html
http://tkinter.fdex.eu/doc/chw.html

In [ ]:
f = Rational(2,3)*(x**3+5)*sqrt(x)

In [ ]:
f